In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "Qwen/Qwen2.5-3B-Instruct"


################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False




################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/DATA/rohan_kirti/niladri/fine_tune/train_dataset/results"

# Number of training epochs
num_train_epochs =1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps =2

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map ='auto'

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset("csv", data_files='/DATA/rohan_kirti/niladri/fine_tune/train_dataset/dataset.csv', split="train")


# # Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

Your GPU supports bfloat16: accelerate training with bf16=True


In [8]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # load_in_8bit=True,
    device_map='auto'
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]


In [9]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
    # packing=packing,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
print("="*10)
print("Starting training...")
trainer.train()
print("Training complete.")
print("="*10)


In [ ]:
new_model = "/DATA/rohan_kirti/niladri/fine_tune/train_dataset/fine_tune model"
print("="*10)
trainer.model.save_pretrained(new_model)
print("Model saved")
print("="*10)

## Saved Fine Tune model

In [1]:
model_name = "Qwen/Qwen2.5-3B-Instruct"
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # low_cpu_mem_usage=True,
    # return_dict=True,
    # torch_dtype=torch.float16,
    device_map='auto',
)

/DATA/rohan_kirti/miniconda3/envs/nilenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.21s/it]


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
input_text = "Hi, I'm looking to get motor insurance for my new electric vehicle. It's a 2024 Tesla Model 3."
inputs = tokenizer(input_text, return_tensors="pt")
input_len = inputs.input_ids.shape[1]

with torch.no_grad():
    outputs = base_model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

generated_tokens = outputs[0, input_len:]
full_response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(full_response)


/DATA/rohan_kirti/miniconda3/envs/nilenv/lib/python3.11/site-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


 Do you have any advice or resources I should check out? Getting motor insurance for your electric vehicle (EV) like the 2024 Tesla Model 3 is a bit different from traditional gasoline vehicles. Here are some key points and resources to consider:

### Key Considerations:
1. **Insurance Types**:
   - **Collision Coverage**: Covers damages to your car in an accident.
   - **Comprehensive Coverage**: Covers damage not caused by collisions, such as theft or vandalism.
   - **Liability Insurance**: Covers damages to other people's property and injuries they may suffer due to your fault.
   
2. **Coverage Levels**:
   - **Minimum Required**: In most states, liability coverage is required. However, it might


In [3]:
from peft iPeftModelmport LoraConfig, 
new_model = "/DATA/rohan_kirti/niladri/fine_tune/train_dataset/fine_tune model_train"

model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

In [4]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [5]:
# Define where to save
save_directory = "/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune"

# Save the merged model (now a plain Hugging Face model)
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)


('/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune/tokenizer_config.json',
 '/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune/special_tokens_map.json',
 '/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune/chat_template.jinja',
 '/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune/vocab.json',
 '/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune/merges.txt',
 '/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune/added_tokens.json',
 '/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune/tokenizer.json')

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
save_directory = "/DATA/rohan_kirti/niladri/fine_tune/train_dataset/qween_fine_tune"
# Load model in FP16 or BF16 or FP32, depending on your hardware
model = AutoModelForCausalLM.from_pretrained(
    save_directory,
    device_map="auto"               # Automatically assigns model to available GPU/CPU
)

tokenizer = AutoTokenizer.from_pretrained(save_directory, trust_remote_code=True)


/DATA/rohan_kirti/miniconda3/envs/nilenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.23s/it]


In [2]:
tokenizer.pad_token = tokenizer.eos_token

In [3]:
input_text = "Hi, I'm looking to get motor insurance for my new electric vehicle. It's a 2024 Tesla Model 3."
inputs = tokenizer(input_text, return_tensors="pt")
input_len = inputs.input_ids.shape[1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

generated_tokens = outputs[0, input_len:]
full_response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(full_response)


/DATA/rohan_kirti/miniconda3/envs/nilenv/lib/python3.11/site-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


 What kind of coverage options do you recommend?
As an AI, I don't have personal experiences or preferences, but I can certainly help you find the best motor insurance options for your 2024 Tesla Model 3. Electric vehicles like yours come with unique features and requirements, so it's essential to choose a policy that caters specifically to them. Here are some recommended coverage options:

1. Comprehensive Coverage: This option covers damage to your Tesla Model 3 from accidents, theft, or natural disasters. It's particularly important for electric vehicles since they have specific components like the battery.

2. Battery Protection: As an electric vehicle, your battery is a significant investment. Comprehensive Battery Protection is a must-have coverage option to ensure your battery is
